---
title: "LC53: Maximum Subarray"
author: "Vahram Poghosyan"
date: "2024-05-23"
categories: ["Leetcode", "Algorithms", "Dyanmic Progamming"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Problem Statement

Given an integer array `nums`, find the subarray with the largest sum, and return its sum.

 
**Example 1**

```
Input: nums = [-2,1,-3,4,-1,2,1,-5,4]
Output: 6
```
**Explanation** 

The subarray `[4,-1,2,1]` has the largest sum `6`.

**Example 2**

```
Input: nums = [1]
Output: 1
````

**Explanation**

The subarray `[1]` has the largest sum `1`.

# Brute-Force Solution

Consider each subarray. It's easy to see that this leads to time complexity $O(n^3)$. There are `n` subarrays of size `1`, `n-1` subarrays of size `2`, ..., and `1` subarray of size `n` (the entire array itself). In addition, for each subarray, computing the sum over it takes $O(n)$ time. So, the leading term of the total time complexity is cubic in `n`.

The code for the brute force solution will not be provided. 

# Optimal Solution

We can solve this problem in a single pass, achieving $O(n)$ complexity by using [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) with **tabulation**. This solution is known by the name of [Kadane's algorithm](https://en.wikipedia.org/wiki/Maximum_subarray_problem#Kadane's_algorithm). 

## Similarity to Other Subarray Problems

The **Maximum Subarray** problem is similar to the **Best Time to Buy and Sell Stock** problem (see [post](lc121_best_time_to_buy_and_sell_stock.ipynb)) and a number of other subarray problems. All of them lends themselves to an iterative, tabulated DP solution. In fact, the [Best Time to Buy and Sell Stock](./lc121_best_time_to_buy_and_sell_stock.ipynb) solution closely resembles Kadane's algorithm (even on sight). The similarities between these two problems are due to the fact that both problems are concerned with finding some *optimal score* over a *contiguous* subarray. Whereas Kadane's is concerned with the subarray having *maximum sum*, the (nameless) algorithm that solves [Best Time to Buy and Sell Stock](./lc121_best_time_to_buy_and_sell_stock.ipynb) is concerned with *maximum profit* (which is defined as the difference between the last element of the optimal subarray and the first one). The score function (how it determines optimality), and the contiguity requirement are what give rise to the key optimal sub-structures in both problems.  

We can prove the correctness of Kadane's algorithm by *proving* the optimal sub-structures (which are simply stated in this post), and by using *loop invariants* to show that the overall answer must be correct. But, we will omit the proof here. Instead, in this post, we will give the intuition for how one might derive the algorithm from scratch. To see an example of a proof, check out the [Best Time to Buy and Sell Stock post](./lc121_best_time_to_buy_and_sell_stock.ipynb), where we don't spend any time solving the problem and, instead, offer its proof of correctness. The proof of correctness of Kadane's algorithm will be very similar to the proof offered in that complementary post (and it is left as an exercise).

## Kadane's Algorithm

### Dynamic Programming - The Key Idea

The key idea behind dynamic programming is to solve sub-problems once, store their results (one way or another), and use them to solve the larger problem. Usually there's some optimal sub-structure to DP problems that make this type of optimization possible -- optimal sub-structure that may reveal itself in the process of walking through an example. Optimal sub-structures are just computationally cheap steps to obtain the solution to the larger problem from that of its sub-problems.  

#### Two Approaches

| Approach | Implementation details |
|----------|-------------|
| **Top-Down (Memoization)**   | Uses **recursion** to solve the problem, and **memoization** to store the solutions to the sub-problems |
| **Bottom-Up (Tabulation)**:   | **Iteratively** solves sub-problems, in a specific order, eliminating the need for recursion (this technique is also known as **tabulation**)|

### Applying DP to the Maximum Subarray Problem

Kadane's algorithm, which solves this problem in a single pass, relies on two top-down, optimal sub-structures that exist within this problem. Once uncovered, these will give us immediate clues into how to write a recursive solution (but this will be left as exercise). In this post we will give the iterative, bottom-up, and tabulated algorithm known as Kadane's Algorithm. 
It's worth noting that one of the optimal sub-structures in this problem stems from the contiguity requirement (of subarrays), which explains why a set of algorithms *like* Kadane's solve a variety of other subarray-related problems (for example, [Best Time to Buy and Sell Stock](./lc121_best_time_to_buy_and_sell_stock.ipynb)). These optiomal sub-structures, ultimately, lead to a *quadratic* reduction in the total number of passes required over subarrays when computing the answer. 

First, let's break the overall problem down into its constituent sub-problems. The sub-problems will be solved in a particular order such that the solution of one, stored in memory, obtains the solution to the next sub-problem (and so on). We shall see that, in fact, this approach ends up not requiring us to do *any* sums over *any* subarrays at all! There are several ways of breaking a problem down array problems into sub-problems using clever indexing.

One way is to observe that any subarray ends (or begins) at some index `k`. We may define a pure abstraction: *The solution to the problem of size-`k+1`* (i.e. index `0` corresponds to a problem of size `1`). Note that this definition isn't pulled out of thin air, it's the global maximum for the `k+1`-sized sub-problem (i.e. the solution to the overall problem were it to have size `k+1`). So, let's call it by its name: `global_max[k]` (where `k` is an index, not an iteration).

We can define another such useful abstraction: *The solution to the problem, if the optimal subarray was constrained to those subarrays which end on index `k`*. Note that this, also, isn't pulled out of thin air. It's the local maximum at index `k` (i.e. the best guess for the answer while lacking global context). So, we call it `local_max[k]`.

Note that `global_max[k]` and `local_max[k]` aren't the same thing. It's easy to get lost in trying to solve this problem by conflating `local_max` with `global_max`, but they're not the same. They're also the only variables needed to implement a single-pass solution, as we'll soon see. Let's see how these two definitions differ.

There may be an input array for which the best `local_max[k]` can achieve are negative sums. For instance, take `k=3` in the following Manim animation (which demonstrates such a case). First, let's import `manim`...

In [1]:
#| code-fold: false
from manim import *

The following is the Manim code that generates the animation below. It's collapsed, because it's not the focus of this post.

In [ ]:
%%manim -qh KadaneAlgoDemo

class KadaneAlgoDemo(Scene):
    def construct(self):    
        # Create table
        array=[[1,-2,3,-5,2,3]]
        table = IntegerTable(array, include_outer_lines=True)
        self.play(Write(table))
        # Get table out of the way...
        self.wait(1)
        self.play(table.animate.scale(0.25)) 
        self.play(table.animate.to_corner(UP + LEFT))

        # Positioning of subarrays
        h_shift = 6.45
        v_shift = 0.5

        # Print subarray(s) ending at index 0
        index = 0
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i]    
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 3 + 1*0.20
        v_shift = 0.5 # Reset vertical shift

        # Print subarray(s) ending at index 1
        index = 1
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i]           
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
            for idx in range(index - left):
                subtable.add_highlighted_cell((1,idx + 1), color=YELLOW_D)
                subtable.get_entries((1, idx + 1)).set_color(BLACK)  # Darken the color of the text against the lighter background
            subtable.add_highlighted_cell((1, index - left + 1), color=BLUE_E)
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 3 + 2*0.20
        v_shift = 0.5 # Reset vertical shift

        # Print subarray(s) ending at index 2
        index = 2
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
            for idx in range(index - left):
                subtable.add_highlighted_cell((1,idx + 1), color=YELLOW_D)
                subtable.get_entries((1, idx + 1)).set_color(BLACK)  # Darken the color of the text against the lighter background
            subtable.add_highlighted_cell((1, index - left + 1), color=BLUE_E)
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 3 + 3*0.20
        v_shift = 0.5 # Reset vertical shift

        # Print subarray(s) ending at index 3
        index = 3
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
            for idx in range(index - left):
                subtable.add_highlighted_cell((1,idx + 1), color=YELLOW_D)
                subtable.get_entries((1, idx + 1)).set_color(BLACK)  # Darken the color of the text against the lighter background
            subtable.add_highlighted_cell((1, index - left + 1), color=BLUE_E)
        self.wait(1)

        # Positioning of subarrays
        h_shift = 5 # Reset horizontal shift
        v_shift = 3 # Reset vertical shift

        # Print subarray(s) ending at index 4
        index = 4
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
            for idx in range(index - left):
                subtable.add_highlighted_cell((1,idx + 1), color=YELLOW_D)
                subtable.get_entries((1, idx + 1)).set_color(BLACK)  # Darken the color of the text against the lighter background
            subtable.add_highlighted_cell((1, index - left + 1), color=BLUE_E)
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 5 + 4*0.20
        v_shift = 3 # Reset vertical shift

        # Print subarray(s) ending at index 4
        index = 5
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
            for idx in range(index - left):
                subtable.add_highlighted_cell((1,idx + 1), color=YELLOW_D)
                subtable.get_entries((1, idx + 1)).set_color(BLACK)  # Darken the color of the text against the lighter background
            subtable.add_highlighted_cell((1, index - left + 1), color=BLUE_E)
        self.wait(5)

::: {#fig-kadane-manim}
![Kadane's Demo](./media/videos/leetcode/1080p60/KadaneAlgoDemo.mp4)
:::

Ignore the highlighted cells for now, we will explain what the colors designate later.

In the case above, `global_max[k]` actually retains its previous value (`global_max[k-1]`), totally omitting the `k`-th cell from consideration. But the word *Sum* in the video is closely related to `local_max[k]`. For a given index `k`, `local_max[k]` is just the greatest of the *Sum* values for that index. This spells out the following optimal sub-structure:


```
global_max[k] = max(global_max[k-1], global_max[k-1] + local_max[k])
```

Note that this is a greedy-choice update for `global_max`, because it asserts that taking the `local_max` step at each turn yields the answer. Note also that it stems from the optimality (i.e. scoring) function being the sum in this problem. In other, similar, problems (like [Best Time to Buy and Sell Stock](./lc121_best_time_to_buy_and_sell_stock.ipynb) the scoring function of that problem may yield another optimal sub-structure that can be exploited. 

Notice that this optimal sub-structure gives us a shortcut by which to update `global_max[k]`, if we already have `local_max[k]` for free (by some dark magic). If we only had a way to go from `local_max[k-1]` to `local_max[k]`, we'd get to the final solution very quickly. We could start with `global_max[0]`, the solution to the base sub-problem which is trivially known, and substitute in this way, in a single pass, until we got to `global_max[n]` (for some `n`-sized problem). Luckily for us, there *is* such a cheap rule that obtains `local_max[k]` from `local_max[k-1]`. It is, again, the result of an optimal sub-structure stemming from the contiguity requirement.

For any given index `k`, to obtain `local_max[k]`, we need not compute the sums of all the subarrays ending at index `k`. Since we know `local_max[k-1]` from solving the previous sub-problem, the update is simply:

```
local_max[k] = max(local_max[k-1] + nums[k], nums[k])
```

To convince ourselves of this, let's watch the Manim video again and let's pay close attention to the yellow and blue cells. The video is nothing but the brute-force approach laid out bare. The cell highlights in the video expose the above optimal sub-structure. In general, sub-structures tend to reveal themselves during the brute-force stage so always start algorithmic problems by working through an example manually. In the video, you’ll notice that any subarray ending at index `k` can be divided into two parts, a subarray ending at index `k-1` (highlighted in yellow) and the single-element subarray `nums[k]` (in blue). Since we know `local_max[k-1]`, to find out `local_max[k]` we no longer need to compute the sums of all the possible subarrays ending at `k-1` -- That's a problem we've solved already. Rather, `local_max[k]` will always be either `nums[k]` itself, or the sum `nums[k] + local_max[k-1]`. So we can do away with much of the summation from the brute force approach.
 
Let's walk through a potential solution that uses these two optimal sub-structures. Because we know `nums[0]` and `local_max[0]` (trivially), we can find out `local_max[1]` by using the expression for `local_max[k]`. And, since we have `global_max[0]` (again, trivially), we can find `global_max[1]` by using the expression for `global_max[k]`. It's easy to see that proceeding iteratively in this manner, by first updating `local_max[k]` then `global_max[k]`, we can arrive at the solution to the overall `n`-sized problem (i.e. `global_max[n]`) without the need to compute sums over subarrays at all!  

## Final Solution

Now we just need to implement a loop that updates `local_max` and `global_max` in this particular manner. For more involved DP problems, we may need a matrix (or some other higher-dimensional data structure) to store the solutions to the sub-problems. But for this problem, two loop invariants (`local_max` and `global_max`) will suffice. Let's write the solution.

In [ ]:
#| code-fold: false
nums = [-2,1,-3,4,-1,2,1,-5,4]

In [4]:
#| code-fold: false
local_max = float('-inf')
global_max = float('-inf')

for i, num in enumerate(nums):
    if num > local_max + num:
        local_max = num
    else:
        local_max = local_max + num
    if global_max < local_max:
        global_max = local_max

print(global_max)

6


# Conclusion

In this post, we explored the Maximum Subarray problem and demonstrated how to solve it using Kadane's Algorithm. We started by understanding the problem statement and examining a brute-force solution, which was computationally expensive with a time complexity of $O(n^3)$. We then introduced Kadane's Algorithm, a dynamic programming approach that optimizes the solution to run in linear time $O(n)$. We also visualized the brute-force algorithm using a Manim animation, which helped us understand the underlying optimal sub-structures and, from there, grok algorithm.